In [ ]:
import pandas as pd
import os
import time
from tqdm import tqdm
from operator import itemgetter
from neo4j import GraphDatabase
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_community.vectorstores import Neo4jVector
from dotenv import load_dotenv
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


load_dotenv()

# Настройки
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USER = os.getenv("NEO4J_USERNAME", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "password")
BENCHMARK_FILE = "benchmark_qa.csv"  # Файл из предыдущего шага
RESULTS_FILE = "benchmark_results.csv"


driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
llm = ChatOllama(model="qwen3:8b", temperature=0)
evaluator_llm = ChatOllama(model="qwen3:8b", temperature=0) 
embeddings = OllamaEmbeddings(
    model="qwen3-embedding:0.6b"
)

# 1. Настройка модели эмбеддингов (должна совпадать с той, что будет при поиске)
print("⏳ Начинаю создание векторного индекса и вычисление эмбеддингов...")
print("Это может занять время, если компаний много...")

vector_store = Neo4jVector.from_existing_index(
    embedding=embeddings,
    index_name="global_knowledge_index",
    url=NEO4J_URI,
    username=NEO4J_USER,
    password=NEO4J_PASSWORD,
    node_label="Searchable",
    text_node_property="description",
    embedding_node_property="embedding"
)

retriever = vector_store.as_retriever(search_kwargs={"k": 3})

⏳ Начинаю создание векторного индекса и вычисление эмбеддингов...
Это может занять время, если компаний много...


In [16]:
rag_prompt = ChatPromptTemplate.from_template("""
Ты финансовый ассистент. Ответь на вопрос, основываясь ТОЛЬКО на предоставленном контексте.
Если информации в контексте нет, ответь "I don't know based on the context".

<context>
{context}
</context>

Question: {input}
""")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Эта магия LCEL делает следующее:
# 1. Принимает {"input": "вопрос"}
# 2. Параллельно:
#    - Ищет документы через retriever -> кладет в ключ 'context'
#    - Прокидывает вопрос -> кладет в ключ 'input'
# 3. .assign вычисляет ключ 'answer', используя контекст и вопрос
rag_chain = RunnableParallel(
    {"context":  itemgetter("input") | retriever, "input": itemgetter("input")}
).assign(
    answer=(
        RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
        | rag_prompt
        | llm
        | StrOutputParser()
    )
)

print("✅ RAG-цепочка собрана на LCEL.")

✅ RAG-цепочка собрана на LCEL.


In [17]:
# --- 3. Логика Оценщика (LLM-as-a-Judge) ---

def evaluate_answer(question, ground_truth, prediction):
    """
    Просит LLM сравнить предсказанный ответ с правильным.
    Возвращает: TRUE (если смысл совпадает) или FALSE.
    """
    # Промпт для судьи
    eval_prompt = f"""
    You are a strict evaluator. Compare the predicted answer with the ground truth.
    
    Question: {question}
    Ground Truth: {ground_truth}
    Prediction: {prediction}
    
    Task: Does the Prediction match the factual meaning of the Ground Truth? 
    Ignore minor phrasing differences.
    If the prediction says "I don't know", mark it as FALSE.
    
    Respond with exactly one word: TRUE or FALSE.
    """
    
    try:
        response = evaluator_llm.invoke(eval_prompt)
        content = response.content.strip().upper()
        return "TRUE" in content
    except:
        return False

# --- 4. Запуск Бенчмарка ---

def run_benchmark():
    if not os.path.exists(BENCHMARK_FILE):
        print(f"Файл {BENCHMARK_FILE} не найден! Сначала сгенерируйте вопросы.")
        return

    df = pd.read_csv(BENCHMARK_FILE)
    print(f"🚀 Запуск теста на {len(df)} вопросах...")

    results = []
    correct_count = 0

    # Используем tqdm для отображения прогресса
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        question = row['question']
        ground_truth = str(row['ground_truth'])
        
        start_time = time.time()
        
        try:
            # 1. Запуск RAG
            response = rag_chain.invoke({"input": question})
            prediction = response['answer']
            
            # Извлекаем найденные документы для анализа
            retrieved_docs = [doc.page_content for doc in response['context']]
            context_text = " || ".join(retrieved_docs) # Сохраняем контекст одной строкой
            
            # 2. Оценка
            is_correct = evaluate_answer(question, ground_truth, prediction)
            
            if is_correct:
                correct_count += 1
                
        except Exception as e:
            prediction = f"ERROR: {e}"
            is_correct = False
            context_text = ""

        elapsed_time = time.time() - start_time

        # Сохраняем результат
        results.append({
            "question": question,
            "ground_truth": ground_truth,
            "prediction": prediction,
            "is_correct": is_correct,
            "retrieved_context": context_text, # Важно видеть, что нашел ретривер
            "time_sec": round(elapsed_time, 2),
            "question_type": row.get('question_type', 'unknown')
        })

    # --- 5. Анализ результатов ---
    
    results_df = pd.DataFrame(results)
    results_df.to_csv(RESULTS_FILE, index=False)
    
    accuracy = (correct_count / len(df)) * 100
    
    print("\n" + "="*30)
    print(f"🏁 БЕНЧМАРК ЗАВЕРШЕН")
    print(f"📊 Точность (Accuracy): {accuracy:.2f}%")
    print(f"💾 Результаты сохранены в: {RESULTS_FILE}")
    print("="*30)
    
    # Вывод проблемных мест (топ-5 ошибок)
    print("\nПримеры ошибок:")
    errors = results_df[results_df['is_correct'] == False].head(5)
    for i, row in errors.iterrows():
        print(f"Q: {row['question']}")
        print(f"Truth: {row['ground_truth']}")
        print(f"Pred: {row['prediction']}")
        print("-" * 20)

if __name__ == "__main__":
    run_benchmark()

🚀 Запуск теста на 100 вопросах...


100%|██████████| 100/100 [18:56<00:00, 11.36s/it]


🏁 БЕНЧМАРК ЗАВЕРШЕН
📊 Точность (Accuracy): 42.00%
💾 Результаты сохранены в: benchmark_results.csv

Примеры ошибок:
Q: What is the current market capitalization of TSCO?
Truth: 28121638912
Pred: I don't know based on the context.
--------------------
Q: Where is the headquarters of American Tower Corporation (REI located?
Truth: Boston, MA
Pred: I don't know based on the context.
--------------------
Q: What is the official website for AbbVie Inc.?
Truth: https://www.abbvie.com
Pred: I don't know based on the context.
--------------------
Q: How many full-time employees work for Workday, Inc.?
Truth: 20588.0
Pred: I don't know based on the context.
--------------------
Q: What is the stock ticker symbol for Baxter International Inc.?
Truth: BAX
Pred: I don't know based on the context.
--------------------
